# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,paradip garh,20.3167,86.6167,85.53,52,41,9.35,IN,1713978892
1,1,inverloch,-38.6333,145.7167,56.30,80,77,16.91,AU,1713978894
2,2,isangel,-19.5500,169.2667,79.14,89,88,5.95,VU,1713978895
3,3,laredo,27.5064,-99.5075,80.89,69,100,11.50,US,1713978671
4,4,thompson,55.7435,-97.8558,41.16,36,20,3.44,CA,1713978854


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_df = city_data_df[(city_data_df['Max Temp'] >= 65) & (city_data_df['Max Temp'] <= 85)] 

# Drop any rows with null values
filtered_city_df = filtered_city_df.dropna()

# Display sample data
filtered_city_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,isangel,-19.5500,169.2667,79.14,89,88,5.95,VU,1713978895
3,3,laredo,27.5064,-99.5075,80.89,69,100,11.50,US,1713978671
7,7,bethel,41.3712,-73.4140,68.90,44,75,9.22,US,1713978895
20,20,holualoa,19.6228,-155.9522,71.44,88,40,6.91,US,1713978900
21,21,taro,-8.3862,115.2800,75.20,100,100,2.21,ID,1713978901
26,26,hawaiian paradise park,19.5933,-154.9731,68.90,83,100,6.91,US,1713978902
29,29,atafu village,-8.5421,-172.5159,84.09,76,60,7.76,TK,1713978614
30,30,gorontalo,0.5412,123.0595,75.58,97,100,2.68,ID,1713978905
32,32,flying fish cove,-10.4217,105.6791,80.28,89,100,6.91,CX,1713978905
37,37,avarua,-21.2078,-159.7750,71.65,83,98,2.30,CK,1713978625


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_to_include = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = city_data_df[columns_to_include].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,paradip garh,IN,20.3167,86.6167,52,
1,inverloch,AU,-38.6333,145.7167,80,
2,isangel,VU,-19.5500,169.2667,89,
3,laredo,US,27.5064,-99.5075,69,
4,thompson,CA,55.7435,-97.8558,36,
...,...,...,...,...,...,...
579,gorele,TR,41.0308,39.0031,48,
580,puerto maldonado,PE,-12.6000,-69.1833,66,
581,jayapura,ID,-2.5333,140.7000,90,
582,al jawf,SA,29.5000,38.7500,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
     "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
    

Starting hotel search
paradip garh - nearest hotel: No hotel found
inverloch - nearest hotel: No hotel found
isangel - nearest hotel: No hotel found
laredo - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
carbonear - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
iskateley - nearest hotel: No hotel found
jackson - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
taro - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel fou

,City,Country,Lat,Lng,Humidity,Hotel Name
0,paradip garh,IN,20.3167,86.6167,52,No hotel found
1,inverloch,AU,-38.6333,145.7167,80,No hotel found
2,isangel,VU,-19.5500,169.2667,89,No hotel found
3,laredo,US,27.5064,-99.5075,69,No hotel found
4,thompson,CA,55.7435,-97.8558,36,No hotel found
...,...,...,...,...,...,...
579,gorele,TR,41.0308,39.0031,48,No hotel found
580,puerto maldonado,PE,-12.6000,-69.1833,66,No hotel found
581,jayapura,ID,-2.5333,140.7000,90,No hotel found
582,al jawf,SA,29.5000,38.7500,14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    hover_cols=["Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name)